# Notebook 3: Model Selection and Training

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import  mean_squared_error, r2_score
import joblib

import math

from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten

from scikeras.wrappers import KerasRegressor
import keras_tuner

from scipy.special import logit, expit
from utils import logit_func, expit_func

import optuna
import random
from datetime import datetime

import os

# Set max column width to None.
pd.set_option('display.max_colwidth', None)

# Import cleaned dataset.
df = pd.read_csv('df_filtered.csv', index_col = 0)

In this notebook, we evaluate several regression models for predicting pre-snap win probability, comparing linear, tree-based, and neural approaches. 

We clip the range of target values to avoid undefined errors at 0 and 1. We then logit-transform our target to stabilize regression and avoid compression at the extremes of 0 and 1. The subsequent inverse transform allows predictions to be returned as probabilities.

To keep preprocessing reproducible and portable across experiments and deployment, this transformation is encapsulated in logit_func and expit_func within utils.py, and then wrapped in TransformedTargetRegressor. These ensure consistent handling of edge cases [0, 1] during both training and prediction.

## Preprocessing

In [3]:
features_to_encode = [
    'down'
]

features_to_scale = [
    'yardline_100_home',
    'ydstogo',
    'home_score_differential',
    'home_spread_line'
]

passthrough_features = [
    'home_pos', 
    'time_weight', 
    'home_timeouts_remaining',
    'away_timeouts_remaining']

target = ['vegas_home_wp']

preprocessing = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(drop='first', sparse_output=False), features_to_encode),
        ('scaler', StandardScaler(), features_to_scale),
        ('noop', FunctionTransformer(validate=False), passthrough_features)  
    ],
    verbose_feature_names_out=False
).set_output(transform='pandas')


X = df.drop('vegas_home_wp', axis = 1)
y = df['vegas_home_wp']


## Model Selection

Ridge/Lasso: These are linear models; regularization tests whether a simple linear approximation can capture win probability dynamics.

In [ ]:
# Ridge.


ridge_params = {
    'model__alpha': np.logspace(-3, 3, 20),
    'model__fit_intercept': [True, False],
    'model__max_iter': [1000, 5000, 10000],
    'model__tol': [1e-4, 1e-3, 1e-2],
    'model__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sag', 'saga'],
    # 'model__solver': ['auto', 'svd', 'cholesky', 'sag', 'saga'],
    # 'model__positive': [True, False],
    'model__copy_X': [True, False]
}


ridge_model = Ridge(random_state = 42)

ridge_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('model', ridge_model)
])

ridge_search = RandomizedSearchCV(
        ridge_pipeline,
        ridge_params,
        n_iter=100,
        cv=3,
        scoring = 'neg_mean_squared_error',
        verbose = 3,
        random_state = 42
    )

# Transformed target regressor with logit/expit transformation.
ridge_model = TransformedTargetRegressor(
    regressor = ridge_search,
    func = logit_func,
    inverse_func = expit_func
)


ridge_model.fit(X, y)
print(f'Ridge best score: {-ridge_model.regressor_.best_score_}')
print(f'Ridge best params: {ridge_model.regressor_.best_params_}')

In [ ]:
# Lasso.

lasso_params = {
    'model__alpha': np.logspace(-4, 1, 20),
    'model__max_iter': [1000, 5000, 10000],
    'model__fit_intercept': [True, False],
    'model__tol': [1e-4, 1e-3, 1e-2],
    'model__selection': ['cyclic', 'random'],
    'model__positive': [True, False],
    'model__warm_start': [True, False]
}


lasso_model = Lasso(random_state = 42)

lasso_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('model', lasso_model)
])


lasso_search = RandomizedSearchCV(
        lasso_pipeline,
        lasso_params,
        n_iter=100,
        cv=3,
        scoring = 'neg_mean_squared_error',
        verbose = 3, 
        random_state = 42
    )

lasso_model = TransformedTargetRegressor(
    regressor = lasso_search,
    func = logit_func,
    inverse_func = expit_func
)


lasso_model.fit(X, y)
print(f'Lasso best score: {-lasso_model.regressor_.best_score_}')
print(f'Lasso best params: {lasso_model.regressor_.best_params_}')

Random Forest: Tree-based ensemble that captures nonlinearities without heavy tuning, but less efficient than boosting methods.

In [ ]:
# Random Forest.

rf_params = {
            'model__n_estimators': [50, 100, 200, 500],
            'model__max_depth': [None, 10, 20, 30, 50],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__max_features': ['sqrt', 'log2'],
            'model__bootstrap': [True, False]
        }

rf_model = RandomForestRegressor(random_state = 42)

rf_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('model', rf_model)
])

rf_search = RandomizedSearchCV(
        rf_pipeline,
        rf_params,
        n_iter=100,
        cv=3,
        scoring = 'neg_mean_squared_error',
        verbose = 3,
        random_state = 42
    )

rf_model = TransformedTargetRegressor(
    regressor = rf_search,
    func = logit_func,
    inverse_func = expit_func
)


rf_model.fit(X, y)
print(f'Random forest best score: {-rf_model.regressor_.best_score_}')
print(f'Random forest best params: {rf_model.regressor_.best_params_}')

XGBoost: Gradient boosting, strong candidate for tabular data, capable of modeling interactions while controlling overfitting.

In [ ]:
# XGBoost.

xgb_params = {
    'model__n_estimators': [100, 300, 500],
    'model__learning_rate': [.01, .05, .1, .3],
    'model__subsample': [.5, .7, 1],
    'model__colsample_bytree': [.5, .7, 1],
    'model__reg_alpha': np.logspace(-4, 4, 20),
    'model__reg_lambda': np.logspace(-4, 4, 20),
    'model__gamma': [0, .1, .3, 1],
    'model__max_depth': [3, 5, 7, 10]
}

xgb_model = XGBRegressor(random_state = 42)

xgb_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('model', xgb_model)
])

xgb_search = RandomizedSearchCV(
        xgb_pipeline,
        xgb_params,
        n_iter=100,
        cv=3,
        scoring = 'neg_mean_squared_error',
        verbose = 3, 
        random_state = 42
    )

xgb_model = TransformedTargetRegressor(
    regressor = xgb_search,
    func = logit_func,
    inverse_func = expit_func
)


xgb_model.fit(X, y)
print(f'XGBoost best score: {-xgb_model.regressor_.best_score_}')
print(f'XGBoost best params: {xgb_model.regressor_.best_params_}')

Neural network: Explores deep learning's ability to model complex relationships, through it is less interpretable.

GridSearchCV and RandomizedSearchCV do not allow testing with different numbers of units among varying numbers of layers. Keras_tuner.RandomSearch will test 2, 3, 4, and 5 layers, with varying numbers of units in each layer. 

Keras_tuner is not compatible with our pipeline as it does not employ the .fit() method. Therefore we transform X_train and X_test manually, and feed the transformed results into the tuner.

We perform 100 runs with 10 epochs each, and then evaluate the top 5 models. We also repeat this process several times to ensure consistent results.

In [ ]:
# Subset X and y.
X = df.drop('vegas_home_wp', axis = 1)
y = df['vegas_home_wp']

# Explicitly define logit function.
def logit_func(y):
    tol = 1e-5
    return logit(np.clip(y, tol, 1-tol))
def expit_func(y):
    return expit(y)

# Logit-transform target.
y = logit_func(y)

# Train-test split.
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .2,
                                                    random_state = 42)

# Transform data.
X_train_transformed = preprocessing.fit_transform(X_train)
X_test_transformed = preprocessing.transform(X_test)

tf.get_logger().setLevel("ERROR")

# Create function to build keras models.
def build_model(hp):
    keras_model = Sequential()
    
    num_layers = hp.Int('num_layers', 
                        min_value = 2, 
                        max_value = 5)
    
    for i in range(num_layers):
        keras_model.add(Dense(hp.Choice(f'units_{i}', [16, 32, 64, 128, 256, 512, 1024]),
                       activation = 'relu'))
    keras_model.add(Dense(1, activation = 'linear'))

    keras_model.compile(optimizer = 'adam', 
                  loss = 'mse', 
                  metrics = ['mse'])
    return keras_model

# Instantiate tuner with build function.
tuner = keras_tuner.RandomSearch(
    build_model,
    objective = 'mse',
    max_trials = 100,
    overwrite = True
)

# Use tuner to search.
tuner.search(X_train_transformed, 
             y_train, 
             epochs = 10, 
             validation_data = (X_test_transformed, y_test))

# Top models.
top_models = tuner.get_best_models(num_models=5)

# View summary of top models.
for i, model in enumerate(top_models):
    model.build(input_shape = (None, X_train_transformed.shape[1]))
    loss, acc = model.evaluate(X_test_transformed, y_test, verbose = 0)

    print(f"\nModel {i+1} Summary: (Accuracy: {acc:.4f})")
    print(f'test: {loss}')
    model.summary()

After several iterations, the top performing sequential neural network configuration was consistently 128/512/128/512/1. 

We train a neural network with the optimal hyperparameters over 100 epochs.

In [ ]:
# Instantiate model.
model = Sequential()

# Set layers.
model.add(Dense(128, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mse'])

# Train model.
model.fit(X_train_transformed, 
          y_train, 
          batch_size = 32, 
          epochs = 100, 
          validation_data=(X_test_transformed, y_test))

# View summary
model.summary()

### Best MSEs:

Ridge: 1.085    
Lasso: 1.085    
Random Forest: .047   
XGBoost: .021    
Neural Network: .039    

The linear models (Ridge, Lasso) performed poorly, suggesting the relationship between features and win probability is nonlinear. Random forest and neural network improved fit substantially, but XGBoost provided the lowest MSE while remaining efficient and stable across folds. As a result, we select XGBoost as our production model.

We select Optuna for refinement because it explores parameter space more efficiently than grid or random search. This reduced training time while improving performance over baseline RandomizedSearchCV results.


XGBoost best hyperparameters from randomized search:
 - 'model__colsample_bytree': 0.7,
 - 'model__gamma': 0, 
 - 'model__learning_rate': 0.1, 
 - 'model__max_depth': 10, 
 - 'model__n_estimators': 300, 
 - 'model__reg_alpha': 1.623776739188721, 
 - 'model__reg_lambda': 11.288378916846883, 
 - 'model__subsample': 0.5


In [ ]:
# Set random seed for reproducibility.
np.random.seed(42)
random.seed(42)

# Subset X and y.
X = df.drop('vegas_home_wp', axis = 1)
y = df['vegas_home_wp']

# Train-test split.
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .2,
                                                    random_state = 42)
# Define objective.
def objective(trial):
    # Define hyperparameter space.
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', .01, .3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 20.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 20.0),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 20),
        'gamma': trial.suggest_int('gamma', 0, 5),
        'random_state': 42
        
    }
   
    # Instantiate model.
    xgb_model = XGBRegressor(random_state = 42)

    # Wrap model in pipeline.
    model_pipe = Pipeline([
        ('preprocessing', preprocessing),
        ('model', xgb_model)
        ])
        
    
    # Wrap pipeline in transformed target regressor.
    final_model = TransformedTargetRegressor(
        regressor = model_pipe,
        func = logit_func,
        inverse_func = expit_func
        )

    # Extract parameters from dictionary.
    final_model.set_params(
        **{f'regressor__model__{key}':value for key, value in params.items()})

    # Set scoring to cross validation with negative MSE.
    score = cross_val_score(
        model_pipe,
        X_train,
        y_train,
        cv=3,
        scoring='neg_mean_squared_error',
        n_jobs=1
    )

    return float(score.mean())

# Set save path.
cwd = os.getcwd()
db_path = f"sqlite:///{cwd}/xgb_tuning_03.db"

# Random seed in sampler for reproducibility.
sampler = optuna.samplers.TPESampler(seed=42)

# Instantiate study with sampler.
study = optuna.create_study(
    study_name="xgb_tuning_03",
    direction="maximize",
    storage=db_path,
    load_if_exists=True,
    sampler = sampler
    # skip_if_exists=True
)

# Optimize study, 500 iterations.
study.optimize(objective, n_trials = 500)

# View best MSE.
study.best_value

In [ ]:
# View best hyperparameters.
best_params = study.best_params
best_params

In [ ]:
# Explicitly define best hyperparameters.
best_params = {
    'n_estimators': 387,
    'max_depth': 15,
    'learning_rate': 0.06537652719126918,
    'subsample': 0.9286366815201457,
    'colsample_bytree': 0.9793042383096315,
    'colsample_bylevel': 0.9450262365116351,
    'colsample_bynode': 0.9847881824040057,
    'reg_alpha': 0.003750294655665686,
    'reg_lambda': 16.297997163154285,
    'min_child_weight': 7,
    'gamma': 0}

# Print list copy-paste friendly list of hyperparameters.
for k, v in best_params.items():
    print(f'{k} = {v},')

Optuna gave us these hyperparameters: 

 - n_estimators = 387,  
 - max_depth = 15,  
 - learning_rate = 0.06537652719126918,  
 - subsample = 0.9286366815201457,  
 - colsample_bytree = 0.9793042383096315,  
 - colsample_bylevel = 0.9450262365116351,  
 - colsample_bynode = 0.9847881824040057,  
 - reg_alpha = 0.003750294655665686,  
 - reg_lambda = 16.297997163154285,  
 - min_child_weight = 7,  
 - gamma = 0,

In [48]:
X = df.drop('vegas_home_wp', axis = 1)
y = df['vegas_home_wp']

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .2,
                                                    random_state = 42)

# Instantiate XGBoost model with optimal hyperparameters.
xgb_refined_model = XGBRegressor(
    n_estimators = 387,
    max_depth = 15,
    learning_rate = 0.06537652719126918,
    subsample = 0.9286366815201457,
    colsample_bytree = 0.9793042383096315,
    colsample_bylevel = 0.9450262365116351,
    colsample_bynode = 0.9847881824040057,
    reg_alpha = 0.003750294655665686,
    reg_lambda = 16.297997163154285,
    min_child_weight = 7,
    gamma = 0,
    random_state = 42
)

# Model wrapped in pipeline.
model_pipe = Pipeline([
    ('preprocessing', preprocessing),
    ('model', xgb_refined_model)
])


# Wrap pipeline in transformed target regressor.
final_model = TransformedTargetRegressor(
    regressor = model_pipe,
    func = logit_func,
    inverse_func = expit_func
)


# Fit model.
final_model.fit(X_train, y_train)

# Predict.
y_pred = final_model.predict(X_test)

# View results.
print(f'Model R²: {r2_score(y_test, y_pred)}')
print(f'Model MSE: {mean_squared_error(y_test, y_pred)}')

Model R²: 0.9971068400060967
Model MSE: 0.00028795634696128486


Our final model resulted in an R² of .997, meaning the model explains over 99% of the variation in sportsbook win probability, along with an inverse-transformed MSE of .0003.

In [42]:
# Export model and combined dataframe.
joblib.dump(final_model, 'final_model.pkl', compress=3)

df_preout = pd.concat([X, y], axis = 1)

y_pred_full = final_model.predict(X)

df_out = pd.concat([df_preout, pd.Series(y_pred_full, index = df_preout.index, name = 'y_pred')], axis = 1)
df_out.to_csv('df_preds.csv')

## Model Selection Summary

 - Linear models (Ridge, Lasso) underfit, confirming the need for nonlinear methods.
 - Tree-based models (Random Forest, XGBoost) captured interactions effectively; XGBoost achieved the best generalization.
 - Neural networks performed competitively.
 - Optuna hyperparameter tuning improved efficiency and final model performance.

We select XGBoost as the final model due to its stability and efficiency.